1. Bibliografía:
  - El **tema 6** permitirá al alumno familiarizarse con la programación genética (PG) 
  - y, **el mencionado artículo**, con una de las variantes de la PG, denominada Evolución Gramatical (del inglés Grammatical Evolution, GE). 
  - El **capítulo 8** describe distintos mecanismos para sintonizar de forma adaptativa cada uno de los diferentes parámetros de los que consta un algoritmo evolutivo (AE). 
  - El **capítulo 10** describe la forma de hibridar un AE con otros métodos de búsqueda. 
  - Finalmente, en el **capítulo 12**, se muestran distintas estrategias para manejar la existencia de restricciones en problemas de optimización que son abordados mediante AEs.
  
2. Secciones
  - Descripción del problema a resolver
  - Método para resolverlo 
    - se debe analizar la idoneidad o no del uso de GE para resolver el problema planteado
    - se debe incluir la expresión matemática de la función de evaluación finalmente empleada
    - se debe incluir la descripción de los diferentes operadores de inicialización, variación y selección empleados
    - se debe incluir la forma de manejar las restricciones, los mecanismos de control de parámetros utilizados, así como los mecanismos de búsqueda local implementados
  - Los resultados de los distintos experimentos realizados
  - Un análisis y comparación de resultados
  - Una sección de conclusiones
  - Una descripción del código implementado. 

3. Evaluación
  - Sobre la presentación (2/10)
    - Se evaluará especialmente la claridad en la redacción de la memoria y la capacidad de síntesis.  
  - Sobre el manejo de restricciones (1/10)
      - Se valorará la originalidad del mecanismo o mecanismos usados para el manejo de restricciones.
  - Sobre la configuración del algoritmo (2/10)
    - Aquí se valorará el procedimiento seguido por el alumno a la hora de elegir la mejor configuración de parámetros del algoritmo, incluyendo la implementación de mecanismos de control de parámetros adaptativos o auto-adaptativos
  - Sobre la hibridación del algoritmo con técnicas de búsqueda local (1/10)
    - Se valorará la originalidad del mecanismo de búsqueda local utilizado.
  - Sobre el análisis y comparación de resultados, y conclusiones (4/10)
    - Se valorará la forma de interpretar y comparar los diferentes experimentos realizados.
      - Es muy importante que dicha valoración se haga siempre en términos de los índices SR, MBF, AES 
      - y cualquier otra gráfica que considere oportuna como, por ejemplo, los plots de progreso de convergencia. 
    - Finalmente, se valorará la calidad de las conclusiones obtenidas a partir de la interpretación y comparación de resultados.


#1. Descripción del problema a resolver

Repitiendo las indicaciones dadas en el documento de la actividad, el problema consiste en implementar un algoritmo evolutivo para calcular la derivada simbólica de una función 
$$ f:X \subseteq \mathcal{R} \rightarrow \mathcal{R} $$ 

Disponemos de las siguientes dos definiciones:

> **Definición de derivada de una función en un punto**: Sea $X \subseteq \mathcal{R}$ un intervalo abierto. Diremos que $f:X \subseteq \mathcal{R} \rightarrow \mathcal{R}$ es derivable en $x_0 \in X$, denotado por $f'(x_0)$, si existe y es finito el límite:
$$
f'(x_0) = \lim \limits_{h \to 0} \frac{f(x_0+h)-f(x_0)}{h}  \tag{1}
$$

> **Definición de derivada de una función en un intervalo**: Sea $X \subseteq \mathcal{R}$ un intervalo abierto. Diremos que $f:X \subseteq \mathcal{R} \rightarrow \mathcal{R}$ es derivable en el intervalo $[a,b] \subseteq X$, si $f$ es derivable en cada uno de los puntos de dicho intervalo, es decir, si:
$$
f'(x) = \lim \limits_{h \to 0} \frac{f(x+h)-f(x)}{h}, \forall x \in [a,b]  \tag{2}
$$

Suponiendo que $f$ sea derivable en $[a,b]$, el problema de calcular la derivada lo vamos a transformar en un nuevo problema de optimización consistente en encontrar una función $g(x)$ que minimice la expresión:
$$
\min \limits_{g(x)} \frac{1}{b-a}\int_{a}^{b} error[f'(x),g(x)]dx \tag{3}
$$

dónde $f'(x)$ se calcularía utilizando la expresión $(2)$.

No obstante, el problema anterior se puede resolver de forma aproximada discretizando el intervalo de definición, es decir, cambiando el operador integral por un sumatorio:
$$
\min \limits_{g(x)} \frac{1}{N+1}\sum_{i=0}^{N} error_i[f'(a+i*h),g(a+i*h)] \tag{4}
$$
dónde $h=\frac{b-a}{N}$ es la anchura del subintervalo de muestreo para conseguir muestrear $N+1$ puntos en el intervalo $[a,b]$, y $f'(a+i*h)$ viene dado por:
$$
f'(a+i*h)=\frac{f(a+(i+1)*h) - f(a+i*h)}{h}, \forall i \in \{0,1,...,N\} \tag{5}
$$

#2.  Método para resolverlo

##2.1. Breve introducción a la GE

La evolución gramatical codifica un conjunto de números pseudo aleatorios **(codones)** en un cromosoma que consiste en un número variable de genes binarios de 8 bits. 

Estos números se usan para seleccionar una regla apropiada a partir de una definición de gramática con **notación Backus-Naur (BNF)**.

**Una gramática de BNF consiste en** la tupla 
$$  \{N, T, P, S \} $$
dónde:
- N es el conjunto de no terminales, 
- T, el conjunto de terminales
- P, un conjunto de reglas de producción que mapean los elementos de N a T 
- S es un símbolo de inicio que es un miembro de N. 
  
Los no terminales de la gramática se mapean en los terminales de la gramática mediante la aplicación recursiva de las reglas dictadas por los valores de los genes. Al finalizar el proceso de mapeo, el código final producido (fenotipo) está formado sólo por terminales.

Por ejemplo, si consideramos esta gramática BNF:

```
N = { expr, op, pre_op } 
T = { Sin, Cos, Tan, Log, +, -, /, *, X, () } 
S = <expr>
```

Y representamos P por:

```
(1) <expr> ::= <expr> <op> <expr>     (A) 
             | ( <expr> <op> <expr> ) (B)
             | <pre-op> ( <expr> )    (C) 
             | <var>                  (D)
(2) <op> ::= + (A)  
           | - (B) 
           | / (C) 
           | * (D)
(3) <pre-op> ::= Sin (A)  
               | Cos (B) 
               | Tan (C) 
               | Log (D)
(4) <var> ::= X
```

Consideremos la regla (1):

```
(1) <expr> ::= <expr> <op> <expr> 
             | ( <expr> <op> <expr> ) 
             | <pre-op> ( <expr> ) 
             | <var>
```

En este caso, el no terminal puede producir uno de cuatro resultados diferentes, para decidir cuál utilizar nuestro sistema toma el siguiente número aleatorio disponible del cromosoma y, en este caso obtiene el módulo cuatro del número para decidir qué regal de producción toma. 

Cada vez que se tiene que tomar una decisión, se lee otro número pseudo aleatorio del cromosoma, y de esta manera, el sistema atraviesa el cromosoma.

En GE es posible que los individuos se queden sin genes durante el proceso de mapeo, y en este caso hay dos alternativas:
- La primera es declarar al individuo inválido y castigarlos con un valor de fitness adecuado
- La segunda es envolver al individuo y reutilizar los genes. Este es un enfoque bastante inusual en EAs, ya que es completamente posible que ciertos genes se usen dos o más veces. 

**Lo que es crucial, sin embargo, es que cada vez que un individuo en particular es mapeado de su genotipo a su fenotipo, se genera la misma salida. Esto se garantiza por el proceso de mapeo descrito anteriormente.**

##2.2. Idoneidad de GE para resolver el problema

Nuestro problema consiste en encontrar la derivada de una función probando/evolucionando distintas combinaciones de otras dadas.

Según leemos en [grammatical-evolution.org](http://www.grammatical-evolution.org/papers/gp98/node1.html): 

> *GE has proved successful when applied to a symbolic regression problem [Ryan 98a], and finding trigonometric identities [Ryan 98b], here we apply GE to a symbolic integration problem taken from the literature [Koza 92]. This involves finding a function which is an integral of Cos(X)+2X+1. In each of these cases we take a subset of C as our target language which is described in Backus Naur Form definition. A Steady State selection mechanism [Syswerda 89] has been employed and was found to reduce the number of generations required to achieve a correct solution. Using this selection mechanism we reapplied our system to the two problems previously tackled and again found an improvement in performance for both of these problems.*

Por lo tanto, según esta referencia, parece que utilizar una técnica evolutiva basada en Evolución Gramatical podría ser la forma más indicada.

Además, teniendo en cuenta que el fundamente de la evolución gramatical se basa en definir una gramática que representa las formas válidas que pueden adoptar los individuos, y teniendo en cuenta que una función objetivo puede alcanzarse como combinación de operadores, números y otras funciones básicas, parece bastante natural que el problema de encontrar la derivadda de una función pueda abordar con técnicas de GE.

#2.3. Expresión matemática de la función de evaluación

Primero necesitamos una función que nos permita tomar $N$ muestras del valor de la derivada de una función $f$ en un intervalo $d = [a,b]$.

In [99]:
from math import sin, cos, exp, log

def muestrea_derivada(f, d, N):    
    """
    Calcula los valores aproximados de la derivada
    de f en el intervalo d tomando N muestras    
    """
    a,b = d
    h = float(b-a)/N
    return [(f(a + (i+1)*h) - f(a + i*h))/h 
            for i in range(N+1)]

# muestrea_derivada(sin, [0,1], 10)

Necesitamos además poder tomar $N$ muestras de la función dada por el fenotipo de un individuo en un intervalo $d=[a,b]$

In [100]:
def muestrea_fenotipo(fen, d, N):
    """
    Calcula los valores de la función que se 
    corresponde con el fenotipo fen 
    en el intervalo d tomando N muestras    
    """
    a,b = d
    h = float(b-a)/N
    f = eval('lambda x:' + fen)
    return [f(a + i*h) for i in range(N+1)]

# muestrea_fenotipo('cos(x)', [0,1], 10)

Y por último, podemos construir la función que determina el error cometido al considerar un fenotipo $ \text{fen}$ como la derivada de una función $f$.

La función de evaluación viene dada por la expresión (4) indicada en el enunciado de la práctica. Existen diferentes formas de calcular el error; en esta práctica vamos a utilizar el error cuadrático.


In [109]:
def error(l1,l2):
    """
    Calcula el error cuadrático medio
    entre dos listas de números
    """ 
    squares = [(l1[i] - l2[i])**2 
               for i in range(len(l1))]
    return sum(squares)/float(len(l1))
    
# error([4,5],[0,0])

20.5

Así por ejemplom, para el caso de seno y coseno, podemos hacer esta prueba:

In [108]:
l = muestrea_derivada(sin, [0,1], 10)
r = muestrea_fenotipo('cos(x)', [0,1], 10)
print('un error pequeño %', error(l,r)

SyntaxError: unexpected EOF while parsing (<ipython-input-108-1043dbf7d862>, line 3)

Para cada experimento tendremos:
- una gramática BNF que nos permite construir el fenotipo desde el genotipo
- individuos, cuyo genotipo son listas de enteros (codones)
- una función f cuya derivada queremos calcular 
- un intervalo en el que la función es derivable
- una expresión de cálculo del error cometido por cada individuo considerado éste como derivada de la función f

In [40]:
import sys
sys.path.insert(0, 'ponyge/src')

import ponyge as p
import numpy as np

def set_up_grammar(g):
    f = open("grammar.bnf", "w")
    f.write(g)
    f.close()
    
set_up_grammar("""
<expr>   ::= <expr><op><expr> \
           | (<expr><op><expr>) \
           | <pre_op>(<expr>) \
           | <var>
<op>     ::= + | - | * | / 
<pre_op> ::= sin | cos | exp | log
<var>    ::= x | 1.0
""")    

g = p.Grammar('grammar.bnf')

i = 0
while i<50:
    m = g.generate([np.random.randint(0,255) for _ in range(10)])
    if m[0]:
        i = i + 1
        print(m)


('x', 2)
('x', 2)
('cos(1.0)', 4)
('x', 2)
('x', 2)
('exp(x)', 4)
('sin(exp(x))', 6)
('(x-1.0)', 6)
('1.0+x', 6)
('x', 2)
('x', 2)
('sin(1.0)', 4)
('log(log(sin(x)))', 8)
('x', 2)
('x', 2)
('x', 2)
('x', 2)
('sin(x)', 4)
('1.0', 2)
('exp(x)', 4)
('1.0', 2)
('x', 2)
('sin(x-1.0)', 8)
('exp(1.0)', 4)
('x', 2)
('1.0', 2)
('(x+1.0)', 6)
('1.0', 2)
('1.0', 2)
('sin(x)', 4)
('x', 2)
('x', 2)
('1.0', 2)
('1.0', 2)
('x', 2)
('x', 2)
('1.0', 2)
('exp(1.0)', 4)
('x', 2)
('x', 2)
('1.0', 2)
('1.0', 2)
('sin(x)', 4)
('1.0', 2)
('(1.0-1.0)*x', 10)
('cos((1.0+1.0))', 8)
('1.0+1.0', 6)
('x', 2)
('x', 2)
('x', 2)
